# Import Required Libraries

In this cell, we import the standard Python libraries used in the script.

* os, sys: For interacting with the operating system and managing environment variables.
* subprocess: To start the MLFlow UI server as a separate process.
* webbrowser & time: To automatically open the MLFlow UI in a browser after a short delay.
* argparse: For parsing command-line arguments.
* socket: To find free ports (if needed).

In [ ]:
import os
import subprocess
import webbrowser
import time
import argparse
import socket
import sys


# Utility Function to Find a Free Port

This cell defines a helper function find_free_port() that finds an available port on the local machine.
This can be useful when you want to automatically assign a port for the MLFlow UI server if one is not specified.

In [ ]:
def find_free_port():
    """Find a free port on the current host."""
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind(('', 0))
        return s.getsockname()[1]


# Define the Function to Launch the MLFlow UI

This cell defines the key function launch_mlflow_ui() which performs the following tasks:

* Tracking URI Setup: It sets the MLFLOW_TRACKING_URI environment variable to point to the shared MLflow run directory.
* Command Construction: It builds the command for launching the MLFlow UI with specified port and host.
* Browser Launch: If the open_browser flag is true, it starts a separate thread that waits for 2 seconds (allowing the server to start) and then opens the MLFlow UI in the default web browser.
* Process Handling: It runs the MLFlow UI server as a subprocess and waits for it to terminate, handling graceful shutdown on a keyboard interrupt.

In [ ]:
def launch_mlflow_ui(tracking_uri, port=None, host='0.0.0.0', open_browser=True):
    """
    Launch the MLFlow UI server.
    
    Args:
        tracking_uri: MLFlow tracking URI
        port: Port to use (None for auto-select)
        host: Host to bind to
        open_browser: Whether to open a browser window
    
    Returns:
        The subprocess running the MLFlow UI server
    """
    if port is None:
        port = 5000  # Default MLFlow UI port

    # Set the tracking URI environment variable so MLFlow UI knows where to find run data
    os.environ['MLFLOW_TRACKING_URI'] = "file:///shared/mlruns"

    # Construct the MLFlow UI command with the chosen port and host settings
    cmd = ["mlflow", "ui", "--port", str(port), "--host", host]
    
    print(f"Launching MLFlow UI with tracking URI: {tracking_uri}")
    print(f"Server will be available at: http://{host}:{port}")
    
    if open_browser:
        # Open browser after a short delay to allow the server to start
        def open_browser_delayed():
            time.sleep(2)
            url = f"http://localhost:{port}"
            print(f"Opening browser at {url}")
            webbrowser.open(url)
        
        import threading
        browser_thread = threading.Thread(target=open_browser_delayed)
        browser_thread.daemon = True
        browser_thread.start()
    
    # Launch the MLFlow UI server as a subprocess
    process = subprocess.Popen(cmd)
    
    print("\nMLFlow UI server running. Press Ctrl+C to stop.")
    try:
        process.wait()
    except KeyboardInterrupt:
        print("\nStopping MLFlow UI server...")
        process.terminate()
        process.wait()
        print("Server stopped.")
    
    return process


# Define Function to Print SSH Tunnel Instructions

This cell defines print_ssh_tunnel_command(), a helper function that prints out a command for setting up an SSH tunnel. This is particularly useful if the MLFlow UI server is running on a remote machine. The printed command instructs the user on how to forward the server's port to their local machine, enabling easy access via a local browser.

In [ ]:
def print_ssh_tunnel_command(port):
    """Print SSH tunnel command for remote access."""
    print("\n---- For Remote Access ----")
    print("If this server is running remotely, you can access it using SSH tunneling.")
    print("Run this command on your local machine:")
    print(f"  ssh -N -L {port}:localhost:{port} username@remote-server")
    print("Then open http://localhost:{port} in your local browser.")
    print("--------------------------")


# Parse Command-Line Arguments and Launch MLFlow UI

In the final cell, the script checks if it is being run as the main module. It then:

* Parses Command-Line Arguments: Uses argparse to define options for port, host, browser launching, and tracking URI.
* Default Tracking URI: Sets a default tracking URI if one isn’t provided, ensuring that MLFlow knows where to find the run data.
* SSH Tunnel Instructions: If a port is provided, it prints SSH tunneling instructions to help with remote access.
* Launch MLFlow UI: Finally, it calls launch_mlflow_ui() with the parsed parameters to start the MLFlow UI server.
* This setup allows the MLFlow UI server to be launched in a flexible and user-friendly manner, ideal for presentations or remote access demonstrations.

In [ ]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Launch MLFlow UI for presentation")
    parser.add_argument('--port', type=int, default=None,
                        help="Port to run MLFlow UI on (default: auto-select)")
    parser.add_argument('--host', type=str, default="0.0.0.0",
                        help="Host to run MLFlow UI on (default: 0.0.0.0)")
    parser.add_argument('--no-browser', action='store_true',
                        help="Don't open a browser window")
    parser.add_argument('--tracking_uri', type=str, default=None,
                        help="MLFlow tracking URI (default: auto-detect)")
    
    args = parser.parse_args()
    
    # Auto-detect tracking URI if not provided
    if args.tracking_uri is None:
        args.tracking_uri = "file:///shared/mlruns"

    # If a port is specified, print SSH tunneling instructions for remote access
    if args.port:
        print_ssh_tunnel_command(args.port)
    
    # Launch the MLFlow UI with the specified parameters
    launch_mlflow_ui(
        tracking_uri=args.tracking_uri,  # Use the provided tracking URI or default to shared directory
        port=args.port,
        host=args.host,  # Host binding allows remote access if needed
        open_browser=not args.no_browser
    )
